# Subsistence Risk Analysis for Bristol

This notebook performs geospatial analysis to build a subsistence risk layer based on:
- Tree data (vegetation coverage)
- Building data (infrastructure)
- Soil data (ground conditions)

The analysis combines these datasets to identify areas with potential subsistence risk.

## 1. Setup and Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import osmnx as ox
from shapely.geometry import Point, Polygon, box
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries imported successfully!")

## 2. Define Study Area

Define the Bristol study area coordinates.

In [ ]:
# Bristol city center coordinates
BRISTOL_CENTER = (51.4545, -2.5879)
BRISTOL_BOUNDS = {
    'north': 51.5200,
    'south': 51.4000,
    'east': -2.5000,
    'west': -2.7000
}

# Create bounding box geometry
bbox_geometry = box(
    BRISTOL_BOUNDS['west'], 
    BRISTOL_BOUNDS['south'],
    BRISTOL_BOUNDS['east'], 
    BRISTOL_BOUNDS['north']
)

print(f"Study area centered at: {BRISTOL_CENTER}")
print(f"Bounding box: {BRISTOL_BOUNDS}")

## 3. Data Collection from APIs

### 3.1 Collect Tree Data from OpenStreetMap

In [ ]:
def collect_tree_data(north, south, east, west):
    """
    Collect tree and vegetation data from OpenStreetMap.
    
    Parameters:
    -----------
    north, south, east, west : float
        Bounding box coordinates
    
    Returns:
    --------
    geopandas.GeoDataFrame
        Tree and vegetation features
    """
    try:
        # Get natural features (trees, forests, parks)
        tags = {
            'natural': ['tree', 'wood', 'tree_row'],
            'landuse': ['forest', 'grass', 'meadow']
        }
        
        trees_gdf = ox.features_from_bbox(
            north, south, east, west, 
            tags=tags
        )
        
        print(f"Collected {len(trees_gdf)} tree/vegetation features")
        return trees_gdf
    
    except Exception as e:
        print(f"Error collecting tree data: {e}")
        # Return empty GeoDataFrame if collection fails
        return gpd.GeoDataFrame()

# Collect tree data
tree_data = collect_tree_data(
    BRISTOL_BOUNDS['north'],
    BRISTOL_BOUNDS['south'],
    BRISTOL_BOUNDS['east'],
    BRISTOL_BOUNDS['west']
)

if not tree_data.empty:
    print(f"\nTree data shape: {tree_data.shape}")
    print(f"Tree data columns: {tree_data.columns.tolist()[:10]}...")

### 3.2 Collect Building Data

In [ ]:
def collect_building_data(north, south, east, west):
    """
    Collect building data from OpenStreetMap.
    
    Parameters:
    -----------
    north, south, east, west : float
        Bounding box coordinates
    
    Returns:
    --------
    geopandas.GeoDataFrame
        Building features
    """
    try:
        # Get buildings from OSM
        buildings_gdf = ox.features_from_bbox(
            north, south, east, west,
            tags={'building': True}
        )
        
        print(f"Collected {len(buildings_gdf)} building features")
        return buildings_gdf
    
    except Exception as e:
        print(f"Error collecting building data: {e}")
        return gpd.GeoDataFrame()

# Collect building data
building_data = collect_building_data(
    BRISTOL_BOUNDS['north'],
    BRISTOL_BOUNDS['south'],
    BRISTOL_BOUNDS['east'],
    BRISTOL_BOUNDS['west']
)

if not building_data.empty:
    print(f"\nBuilding data shape: {building_data.shape}")
    print(f"Building data columns: {building_data.columns.tolist()[:10]}...")

### 3.3 Generate Soil Data

Note: For demonstration, we'll generate synthetic soil data. In production, you would integrate with soil data APIs like:
- BGS Geology API
- European Soil Data Centre (ESDAC)
- UK Soil Observatory

In [ ]:
def generate_soil_grid(bounds, grid_size=0.005):
    """
    Generate a grid of soil data points with synthetic properties.
    In production, replace with actual API calls to soil databases.
    
    Parameters:
    -----------
    bounds : dict
        Dictionary with 'north', 'south', 'east', 'west' keys
    grid_size : float
        Size of grid cells in degrees
    
    Returns:
    --------
    geopandas.GeoDataFrame
        Grid of soil sample points with properties
    """
    # Create grid of points
    lons = np.arange(bounds['west'], bounds['east'], grid_size)
    lats = np.arange(bounds['south'], bounds['north'], grid_size)
    
    points = []
    soil_types = ['clay', 'silt', 'sand', 'loam', 'peat']
    
    for lon in lons:
        for lat in lats:
            # Generate synthetic soil properties
            soil_type = np.random.choice(soil_types)
            clay_content = np.random.uniform(10, 60)  # percentage
            moisture = np.random.uniform(20, 80)  # percentage
            shrink_swell = np.random.uniform(1, 10)  # risk score
            
            points.append({
                'geometry': Point(lon, lat),
                'soil_type': soil_type,
                'clay_content': clay_content,
                'moisture': moisture,
                'shrink_swell_potential': shrink_swell
            })
    
    soil_gdf = gpd.GeoDataFrame(points, crs='EPSG:4326')
    print(f"Generated {len(soil_gdf)} soil data points")
    return soil_gdf

# Generate soil data
soil_data = generate_soil_grid(BRISTOL_BOUNDS)
print(f"\nSoil data shape: {soil_data.shape}")
print(f"\nSample soil data:")
print(soil_data.head())

## 4. Data Visualization

### 4.1 Visualize Individual Datasets

In [ ]:
def create_base_map(center, zoom_start=12):
    """
    Create a base folium map.
    
    Parameters:
    -----------
    center : tuple
        (latitude, longitude) for map center
    zoom_start : int
        Initial zoom level
    
    Returns:
    --------
    folium.Map
        Base map object
    """
    m = folium.Map(
        location=center,
        zoom_start=zoom_start,
        tiles='OpenStreetMap'
    )
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# Create base map for Bristol
base_map = create_base_map(BRISTOL_CENTER)
print("Base map created successfully")

In [ ]:
# Visualize tree data
tree_map = create_base_map(BRISTOL_CENTER)

if not tree_data.empty:
    # Sample subset for visualization if dataset is large
    trees_sample = tree_data.sample(min(1000, len(tree_data)))
    
    folium.GeoJson(
        trees_sample.to_json(),
        name='Trees & Vegetation',
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'darkgreen',
            'weight': 1,
            'fillOpacity': 0.4
        }
    ).add_to(tree_map)
    
    folium.LayerControl().add_to(tree_map)
    print(f"Tree map created with {len(trees_sample)} features")
else:
    print("No tree data available for visualization")

# Display the map
tree_map

In [ ]:
# Visualize building data
building_map = create_base_map(BRISTOL_CENTER)

if not building_data.empty:
    # Sample subset for visualization
    buildings_sample = building_data.sample(min(500, len(building_data)))
    
    folium.GeoJson(
        buildings_sample.to_json(),
        name='Buildings',
        style_function=lambda x: {
            'fillColor': 'gray',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.5
        }
    ).add_to(building_map)
    
    folium.LayerControl().add_to(building_map)
    print(f"Building map created with {len(buildings_sample)} features")
else:
    print("No building data available for visualization")

# Display the map
building_map

In [ ]:
# Visualize soil data with heatmap
soil_map = create_base_map(BRISTOL_CENTER)

# Create heatmap data
heat_data = [[point.xy[1][0], point.xy[0][0], val] 
             for point, val in zip(soil_data.geometry, soil_data.shrink_swell_potential)]

plugins.HeatMap(
    heat_data,
    name='Soil Shrink-Swell Potential',
    min_opacity=0.3,
    radius=15,
    blur=20,
    gradient={
        0.0: 'blue',
        0.5: 'yellow',
        1.0: 'red'
    }
).add_to(soil_map)

folium.LayerControl().add_to(soil_map)
print(f"Soil heatmap created with {len(heat_data)} points")

# Display the map
soil_map

### 4.2 Statistical Visualizations

In [ ]:
# Create statistical plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Soil type distribution
if not soil_data.empty:
    soil_data['soil_type'].value_counts().plot(kind='bar', ax=axes[0, 0], color='brown')
    axes[0, 0].set_title('Soil Type Distribution')
    axes[0, 0].set_xlabel('Soil Type')
    axes[0, 0].set_ylabel('Count')

# Clay content distribution
if not soil_data.empty:
    axes[0, 1].hist(soil_data['clay_content'], bins=30, color='sienna', edgecolor='black')
    axes[0, 1].set_title('Clay Content Distribution')
    axes[0, 1].set_xlabel('Clay Content (%)')
    axes[0, 1].set_ylabel('Frequency')

# Shrink-swell potential distribution
if not soil_data.empty:
    axes[1, 0].hist(soil_data['shrink_swell_potential'], bins=30, color='orangered', edgecolor='black')
    axes[1, 0].set_title('Shrink-Swell Potential Distribution')
    axes[1, 0].set_xlabel('Risk Score')
    axes[1, 0].set_ylabel('Frequency')

# Moisture content distribution
if not soil_data.empty:
    axes[1, 1].hist(soil_data['moisture'], bins=30, color='skyblue', edgecolor='black')
    axes[1, 1].set_title('Soil Moisture Distribution')
    axes[1, 1].set_xlabel('Moisture (%)')
    axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('soil_statistics.png', dpi=300, bbox_inches='tight')
plt.show()

print("Statistical plots created and saved")

## 5. Subsistence Risk Layer Computation

Calculate subsistence risk based on multiple factors:
- Soil shrink-swell potential
- Clay content
- Proximity to trees (root damage risk)
- Building density

In [ ]:
def calculate_subsistence_risk(soil_data, tree_data, building_data, grid_size=0.01):
    """
    Calculate subsistence risk layer by combining multiple data sources.
    
    Parameters:
    -----------
    soil_data : geopandas.GeoDataFrame
        Soil characteristics data
    tree_data : geopandas.GeoDataFrame
        Tree/vegetation data
    building_data : geopandas.GeoDataFrame
        Building footprint data
    grid_size : float
        Size of analysis grid cells in degrees
    
    Returns:
    --------
    geopandas.GeoDataFrame
        Grid with computed subsistence risk scores
    """
    print("Calculating subsistence risk layer...")
    
    # Prepare soil risk component (normalize to 0-1)
    soil_risk = soil_data.copy()
    
    # Normalize soil factors
    soil_risk['soil_score'] = (
        (soil_risk['shrink_swell_potential'] / 10.0) * 0.5 +
        (soil_risk['clay_content'] / 100.0) * 0.3 +
        (soil_risk['moisture'] / 100.0) * 0.2
    )
    
    # Create analysis grid
    bounds = BRISTOL_BOUNDS
    lons = np.arange(bounds['west'], bounds['east'], grid_size)
    lats = np.arange(bounds['south'], bounds['north'], grid_size)
    
    risk_grid = []
    
    for i, lon in enumerate(lons):
        for j, lat in enumerate(lats):
            # Create grid cell polygon
            cell = box(lon, lat, lon + grid_size, lat + grid_size)
            cell_center = Point(lon + grid_size/2, lat + grid_size/2)
            
            # Calculate soil risk for this cell
            soil_points = soil_risk[soil_risk.intersects(cell)]
            if len(soil_points) > 0:
                avg_soil_risk = soil_points['soil_score'].mean()
            else:
                avg_soil_risk = 0
            
            # Calculate tree proximity risk (trees can cause subsidence)
            tree_risk = 0
            if not tree_data.empty:
                nearby_trees = tree_data[tree_data.distance(cell_center) < 0.01]  # ~1km
                tree_risk = min(len(nearby_trees) / 10.0, 1.0)  # Normalize
            
            # Calculate building density (buildings at risk)
            building_density = 0
            if not building_data.empty:
                buildings_in_cell = building_data[building_data.intersects(cell)]
                building_density = min(len(buildings_in_cell) / 20.0, 1.0)  # Normalize
            
            # Combined risk score (weighted average)
            risk_score = (
                avg_soil_risk * 0.5 +  # 50% soil factors
                tree_risk * 0.3 +       # 30% tree proximity
                building_density * 0.2  # 20% building density
            )
            
            # Categorize risk
            if risk_score < 0.3:
                risk_category = 'Low'
            elif risk_score < 0.6:
                risk_category = 'Medium'
            else:
                risk_category = 'High'
            
            risk_grid.append({
                'geometry': cell,
                'risk_score': risk_score,
                'risk_category': risk_category,
                'soil_risk': avg_soil_risk,
                'tree_risk': tree_risk,
                'building_density': building_density
            })
    
    risk_gdf = gpd.GeoDataFrame(risk_grid, crs='EPSG:4326')
    print(f"Subsistence risk layer created with {len(risk_gdf)} grid cells")
    
    return risk_gdf

# Calculate risk layer
risk_layer = calculate_subsistence_risk(soil_data, tree_data, building_data)

print(f"\nRisk layer shape: {risk_layer.shape}")
print(f"\nRisk statistics:")
print(risk_layer[['risk_score', 'soil_risk', 'tree_risk', 'building_density']].describe())
print(f"\nRisk category distribution:")
print(risk_layer['risk_category'].value_counts())

## 6. Visualize Combined Risk Layer

In [ ]:
# Create comprehensive risk map
risk_map = create_base_map(BRISTOL_CENTER)

# Define color scheme for risk levels
def get_risk_color(risk_score):
    """Return color based on risk score."""
    if risk_score < 0.3:
        return '#2ecc71'  # Green (low risk)
    elif risk_score < 0.6:
        return '#f39c12'  # Orange (medium risk)
    else:
        return '#e74c3c'  # Red (high risk)

# Add risk layer
for idx, row in risk_layer.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda x, color=get_risk_color(row['risk_score']): {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.5
        },
        tooltip=folium.Tooltip(
            f"Risk: {row['risk_category']}<br>"
            f"Score: {row['risk_score']:.2f}<br>"
            f"Soil: {row['soil_risk']:.2f}<br>"
            f"Trees: {row['tree_risk']:.2f}<br>"
            f"Buildings: {row['building_density']:.2f}"
        )
    ).add_to(risk_map)

# Add legend
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 200px; height: 120px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
<p style="margin-bottom: 5px;"><b>Subsistence Risk</b></p>
<p style="margin: 5px;"><span style="background-color: #2ecc71; padding: 5px;">█</span> Low (< 0.3)</p>
<p style="margin: 5px;"><span style="background-color: #f39c12; padding: 5px;">█</span> Medium (0.3-0.6)</p>
<p style="margin: 5px;"><span style="background-color: #e74c3c; padding: 5px;">█</span> High (> 0.6)</p>
</div>
'''

risk_map.get_root().html.add_child(folium.Element(legend_html))
folium.LayerControl().add_to(risk_map)

print("Risk map visualization created")

# Display the map
risk_map

## 7. Export and Publish Layer

Export the risk layer in multiple formats for sharing and publishing.

In [ ]:
# Save risk layer to GeoJSON (for web applications)
risk_layer.to_file('subsistence_risk_layer.geojson', driver='GeoJSON')
print("Risk layer saved as GeoJSON")

# Save risk layer to Shapefile (for GIS software)
risk_layer.to_file('subsistence_risk_layer.shp')
print("Risk layer saved as Shapefile")

# Save risk map as HTML (interactive web map)
risk_map.save('subsistence_risk_map.html')
print("Interactive risk map saved as HTML")

# Export summary statistics to CSV
risk_summary = risk_layer[[
    'risk_score', 'risk_category', 'soil_risk', 'tree_risk', 'building_density'
]].copy()

# Add geometry centroid for reference
risk_summary['centroid_lon'] = risk_layer.geometry.centroid.x
risk_summary['centroid_lat'] = risk_layer.geometry.centroid.y

risk_summary.to_csv('subsistence_risk_summary.csv', index=False)
print("Risk summary saved as CSV")

print("\n=== Export Complete ===")
print("Files created:")
print("1. subsistence_risk_layer.geojson - GeoJSON format for web apps")
print("2. subsistence_risk_layer.shp (+ .shx, .dbf, .prj) - Shapefile for GIS")
print("3. subsistence_risk_map.html - Interactive web map")
print("4. subsistence_risk_summary.csv - Summary statistics")
print("5. soil_statistics.png - Statistical visualizations")

## 8. Additional Analysis Tools

### 8.1 Query Risk at Specific Location

In [ ]:
def query_risk_at_location(lat, lon, risk_layer):
    """
    Query subsistence risk at a specific location.
    
    Parameters:
    -----------
    lat, lon : float
        Coordinates to query
    risk_layer : geopandas.GeoDataFrame
        Risk layer data
    
    Returns:
    --------
    dict
        Risk information at the location
    """
    point = Point(lon, lat)
    
    # Find grid cell containing the point
    result = risk_layer[risk_layer.contains(point)]
    
    if len(result) > 0:
        row = result.iloc[0]
        return {
            'location': (lat, lon),
            'risk_score': row['risk_score'],
            'risk_category': row['risk_category'],
            'soil_risk': row['soil_risk'],
            'tree_risk': row['tree_risk'],
            'building_density': row['building_density']
        }
    else:
        return {'error': 'Location outside study area'}

# Example query
example_lat, example_lon = BRISTOL_CENTER
risk_info = query_risk_at_location(example_lat, example_lon, risk_layer)

print("Risk query at Bristol city center:")
for key, value in risk_info.items():
    print(f"  {key}: {value}")

### 8.2 Generate Risk Report

In [ ]:
def generate_risk_report(risk_layer):
    """
    Generate a summary report of subsistence risk analysis.
    
    Parameters:
    -----------
    risk_layer : geopandas.GeoDataFrame
        Risk layer data
    
    Returns:
    --------
    str
        Formatted report text
    """
    total_cells = len(risk_layer)
    high_risk = len(risk_layer[risk_layer['risk_category'] == 'High'])
    medium_risk = len(risk_layer[risk_layer['risk_category'] == 'Medium'])
    low_risk = len(risk_layer[risk_layer['risk_category'] == 'Low'])
    
    avg_risk = risk_layer['risk_score'].mean()
    max_risk = risk_layer['risk_score'].max()
    min_risk = risk_layer['risk_score'].min()
    
    report = f"""
    ===================================
    SUBSISTENCE RISK ANALYSIS REPORT
    Bristol Study Area
    ===================================
    
    Analysis Coverage:
    - Total grid cells analyzed: {total_cells}
    - Study area: {BRISTOL_BOUNDS}
    
    Risk Distribution:
    - High Risk areas: {high_risk} ({high_risk/total_cells*100:.1f}%)
    - Medium Risk areas: {medium_risk} ({medium_risk/total_cells*100:.1f}%)
    - Low Risk areas: {low_risk} ({low_risk/total_cells*100:.1f}%)
    
    Risk Score Statistics:
    - Average risk score: {avg_risk:.3f}
    - Maximum risk score: {max_risk:.3f}
    - Minimum risk score: {min_risk:.3f}
    
    Risk Factors:
    - Soil conditions (shrink-swell, clay content, moisture)
    - Tree proximity (root damage potential)
    - Building density (infrastructure at risk)
    
    Outputs Generated:
    - Interactive web map (subsistence_risk_map.html)
    - GeoJSON layer (subsistence_risk_layer.geojson)
    - Shapefile (subsistence_risk_layer.shp)
    - Summary CSV (subsistence_risk_summary.csv)
    - Statistical plots (soil_statistics.png)
    
    ===================================
    """
    
    return report

# Generate and display report
report = generate_risk_report(risk_layer)
print(report)

# Save report to file
with open('subsistence_risk_report.txt', 'w') as f:
    f.write(report)
    
print("Report saved to subsistence_risk_report.txt")

## 9. API Integration Examples

### Placeholder for Additional API Integrations

This section can be extended to integrate with additional data sources:

1. **BGS (British Geological Survey) API**
   - Geological data
   - Ground stability information
   - Historical subsidence records

2. **UK Environment Agency API**
   - Flood risk data
   - Water table levels
   - Environmental factors

3. **Weather API (e.g., Met Office)**
   - Rainfall data
   - Drought conditions
   - Climate patterns

4. **Property Data APIs**
   - Building age and type
   - Foundation information
   - Insurance claims data

In [ ]:
# Example API integration template
def integrate_external_api(api_url, params):
    """
    Template for integrating external APIs.
    
    Parameters:
    -----------
    api_url : str
        API endpoint URL
    params : dict
        API parameters
    
    Returns:
    --------
    dict or geopandas.GeoDataFrame
        API response data
    """
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        data = response.json()
        return data
    except Exception as e:
        print(f"API error: {e}")
        return None

print("API integration template defined")
print("To use: integrate_external_api('your_api_url', {'param': 'value'})")

## 10. Publishing Options

The generated files can be published using:

1. **Web Map (HTML)**: Upload `subsistence_risk_map.html` to any web server
2. **GeoJSON**: Use with web mapping libraries (Leaflet, Mapbox, etc.)
3. **Shapefile**: Import into QGIS, ArcGIS, or other GIS software
4. **Map Tiles**: Convert to raster tiles for high-performance web serving
5. **API Service**: Serve the GeoJSON via a REST API
6. **Cloud Platforms**: 
   - Google Earth Engine
   - ArcGIS Online
   - Mapbox Studio
   - Carto

### Example: Create Leaflet Integration Code

In [ ]:
# Generate sample HTML/JavaScript code for embedding the map
leaflet_code = '''
<!DOCTYPE html>
<html>
<head>
    <title>Bristol Subsistence Risk Map</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css" />
    <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
    <style>
        #map { height: 600px; width: 100%; }
    </style>
</head>
<body>
    <h1>Bristol Subsistence Risk Map</h1>
    <div id="map"></div>
    <script>
        // Initialize map
        var map = L.map('map').setView([51.4545, -2.5879], 12);
        
        // Add base layer
        L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
            attribution: '© OpenStreetMap contributors'
        }).addTo(map);
        
        // Load GeoJSON risk layer
        fetch('subsistence_risk_layer.geojson')
            .then(response => response.json())
            .then(data => {
                L.geoJSON(data, {
                    style: function(feature) {
                        var risk = feature.properties.risk_score;
                        var color = risk < 0.3 ? '#2ecc71' : 
                                   risk < 0.6 ? '#f39c12' : '#e74c3c';
                        return {
                            fillColor: color,
                            weight: 1,
                            opacity: 0.8,
                            color: 'black',
                            fillOpacity: 0.5
                        };
                    },
                    onEachFeature: function(feature, layer) {
                        layer.bindPopup(
                            '<b>Risk Category:</b> ' + feature.properties.risk_category + '<br>' +
                            '<b>Risk Score:</b> ' + feature.properties.risk_score.toFixed(2)
                        );
                    }
                }).addTo(map);
            });
    </script>
</body>
</html>
'''

# Save the code template
with open('leaflet_integration_example.html', 'w') as f:
    f.write(leaflet_code)

print("Leaflet integration example saved to leaflet_integration_example.html")
print("\nThis file demonstrates how to embed the risk layer in a custom web page.")

## Summary

This notebook provides a complete geospatial analysis workflow for subsistence risk assessment:

✓ **Data Collection**: Integrated with OpenStreetMap API for tree and building data, with extensible framework for additional APIs

✓ **Data Processing**: Combined multiple data sources into unified risk layer

✓ **Visualization**: Interactive maps, heatmaps, and statistical plots

✓ **Risk Analysis**: Multi-factor risk scoring based on soil, trees, and buildings

✓ **Export & Publishing**: Multiple output formats (GeoJSON, Shapefile, HTML, CSV)

✓ **Query Tools**: Location-based risk queries and reporting

### Next Steps:
1. Integrate real soil data APIs (BGS, ESDAC)
2. Add historical subsidence data
3. Incorporate climate/weather data
4. Fine-tune risk calculation weights
5. Deploy as web service
6. Add real-time data updates